<img src="../img/crowdlearning-etic.png" alt="Logo ETIC" align="right">

<br>
<h1><font color="#004D7F" size=6>Web Scraping</font></h1>

<br><br>
<div style="text-align: right">
<font color="#004D7F" size=3>Antonio Jesús Gil</font><br>
<font color="#004D7F" size=3>Introducción a la Ciencia de Datos</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>


* [1. Introducción](#section1)
* [2. Páginas web](#section2)
* [3. HTML](#section3)
* [4. Parseando HTML con BeautifulSoup](#section4)
* [5. La librería requests](#section5)
* [6. Web scraping de una página de predicción meteorológica](#section6)
* [7. Web scraping con un navegador real](#section7)
* [Referencias](#referencias)

---

<a id="section1"></a>
# <font color="#004D7F"> 1. Introducción</font>

Cuando trabajamos como científico de datos, es común querer usar datos encontrados en Internet. En muchas ocasiones, seremos capaces de acceder a los datos en CSV o a través de un Application Programming Interface (API). Sin embargo, hay veces que los datos que queremos utilizar sólo están disponibles a través de una página web. En estos casos, podemos utilizar una técnica llamada **web scraping** para **transformar estos datos que obtenemos desde un servidor web en un formato que podamos utilizar en nuestros análisis**.

In [ ]:
!pip install requests
!pip install beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup

---

<a id="section2"></a>
# <font color="#004D7F"> 2. Páginas web</font>

Cuando visitamos una página web, nuestro navegador hace una petición a un servidor web. Esta petición se llama una petición GET, ya que estamos obteniendo ficheros del servidor. El servidor envía de vuelta ficheros que le dicen al navegador como debe renderizar la paǵina. Estas filas pueden ser de tres tipos:

* HTML: Contienen en contenido principal de la página
* CSS: Añaden estilos a la página
* JS: Añaden interactividad a la página — Javascript files add interactivity to web pages.
* Contenido multimedia: Permiten a las páginas mostrar imágenes, videos, etc.

Cuando nuestro navegador recive todos estos ficheros, renderiza la página y nos la muestra. Para esto, un monton de cosas suceden en segundo plano, pero no tenemos que preocuparnos de estos cuando hablamos de web scraping. Cuando hacemos web scraping, estamos interesados sólo en el contenido principal de la página, el HTML.

---

<a id="section3"></a> 
# <font color="#004D7F">3. HTML</font>

HyperText Markup Language (HTML) es el lenguaje con el que se crean las páginas web. HTML no es un lenguaje de programación, es un lenguaje de marcado que le dice al navegador como tiene que renderizar el contenido.

A continuación se muestra una pequeña guía sobre HTML con la información mínima para poder hacer scraping de forma efectiva. HTML consiste en elementos llamados **etiquetas**. La etiqueta más básica es la etiqueta &lt;html&gt;. Esta etiqueta le dice al navegador que todo lo que encontremos dentro es HTML. Podemos crear un documento HTML mínimo usando sólamente esta etiqueta

```html
<html>
</html>
```

Como no hemos añadido ningún contenido a nuestra página aún, si la visualizasemos a través de un servidor web, no veríamos nada todavía.

Dentro de la etiqueta html, ponemos 2 etiquetas más, las etiquetas head y body. El contenido principal de la página se encuentra dentro de la etiqueta body. La etiqueta head contiene el título de la página y otra información que en general no es útil para el web scraping.

```html
<html>
    <head>
    </head>
    <body>
    </body>
</html>
```

Como se muestra en el código anterior, las  etiquetas head y body están dentro del tag html. En HTML las etiquetas se anidan y unas etiquetas pueden colocarse dentro de otras.

Ahora vamos a añadir nuestro primer contenido a la página, en forma de una etiqueta p. La etiqueta p define un parrafo y el texto dentro de la etiqueta se renderizaría como un párrafo en la página.

```html
<html>
    <head>
    </head>
    <body>
        <p>
            Convierte en un científico de datos.
        </p>
        <p>
            Domina el lenguaje de programación que tiene más crecimiento.
        </p>
    </body>
</html>
```

En general usamos los siguientes nombres para referirnos a la posición de una etiqueta en relación a otras etiquetas.
* hija: Una etiqueta hija es una etiqueta que se encuentra dentro de otra etiqueta. En el ejemplo anterior, las 2 etiquetas p son hijas de la etiqueta body.
* padre: Un padre es una etiqueta que tiene otra dentro. En el ejemplo, html es padre de la etiqueta body.
* hermanos: Un hermano es una etiqueta que está incluida dentro del mismo padre que otra etiqueta. Por ejemplo, head y body son hermanas, ya que ambas se encuentran dentro de la etiqueta html. 

También podemos agregar **propiedades** a las etiquetas HTML que cambiar su significado.

```html
<html>
    <head>
    </head>
    <body>
        <p>
            Convierte en un científico de datos.
            <a href="https://www.etics.es">Etic</a>
        </p>
        <p>
            Domina el lenguaje de programación que tiene más crecimiento.
            <a href="https://www.python.org">Python</a>
        </p>
    </body>
</html>
```

En el ejemplo anteior, hemos añadido dos etiquetas a. Las etiquetas a son links y le dicen al navegador que renderice un enlace a otra página. La propiedad href de la etiqueta le dice a donde debe dirigir el enlace.

Otras etiquetas comunes en HTML son las siguientes:

* div: Indica una división o área de la página.
* h1, h2, h3, h4, h5: Encabezados de distinto tamaño/importancia.
* table: Crea una tabla
* form: Crea un formulario de entrada
* img: Una imagen incrustada en el documento

Antes de empezar con el web scraping, es importante ver las propiedas class y id. Estas propiedas especiales le dan a los elementos HTML nombres, y hacen más fácil interactuar con el contenido sobre el que queremos hacer scraping. **Un elemento puede tener múltiples clases, y una clase puede ser compartida por varios elementos**. Cada elemento puede tener una sóla id, y esa id sólo puede ser usada una vez en la página. Las propiedades class e id son opciones, y no todos los elementos las van a tener.

Podemos añadir clases a nuestro ejemplo, quedando de la siguiente forma:

```html
<html>
    <head>
    </head>
    <body>
        <p class="bold-paragraph">
            Convierte en un científico de datos.
            <a href="https://www.etics.es" id="course-link">Etic</a>
        </p>
        <p class="bold-paragraph extra-large">
            Domina el lenguaje de programación que tiene más crecimiento.
            <a href="https://www.python.org" class="extra-large">Python</a>
        </p>
    </body>
</html>
```

---

<a id="section4"></a> 
# <font color="#004D7F">4. Parseando HTML con BeautifulSoup</font>


Para parsear documentos HTML vamos a utilizar la librería BeautifulSoup.

In [ ]:
#!pip install beautifulsoup4
!pip show beautifulsoup4

Para utilizar BeautifulSoup, lo primero que debemos hacer es crear un objeto de tipo BeautifulSoup. 

El constructor de BeautifulSoup puede aceptar dos argumentos. El primer argumento es el código HTML (o XML), y el segundo argumento es el parser que se quiere usar.

In [ ]:
from bs4 import BeautifulSoup
 
soup = BeautifulSoup("<html><head></head><body><p>Curso de ciencia de datos</p></body></html>", "html.parser")

In [ ]:
print(soup.prettify())

## <font color="#004D7F">Parsers para BeautifulSoup</font>

El `html.parser` es un parser integrado en BeautifulSoup, y no funciona en versiones antiguas de Python. Otros parsers que se pueden utilizar son `lxml` y `html5lib`.

In [ ]:
!pip install lxml
!pip install html5lib

In [ ]:
#!conda install -y lxml

In [ ]:
#!pip uninstall -y lxml

El parser `lxml` es muy rápido y es el más utilizado. Por el otro lado, el parser `html5lib` es muy lento, pero también extremadamente indulgente.

In [ ]:
html = "<html><p>This is <b>invalid HTML</p></html>"
# Html parser está obsoleto aunque a veces sea de utiliad
soup = BeautifulSoup(html, "html.parser")
#print(soup)

# Es capaz de add etiquetas
# Es la más rapida de las 3 y la que cumple todos los escenarios (compliance)
soup = BeautifulSoup(html, "lxml")
#print(soup)

soup = BeautifulSoup(html, features="lxml")
print(soup.prettify())

## <font color="#004D7F">Primeros pasos con BeautifulSoup</font>

Empezaremos parseando un pequeño código HTML para ir viendo las funcionalidades de la librería.


In [ ]:
html='''
<!DOCTYPE html>
<html>
 <head>
  <title>
   ETIC: Primeros pasos con BeautifulSoup
  </title>
 </head>
 <body>
  <p>
   Primer parrafo de la página
  </p>
 </body>
</html>
'''

soup = BeautifulSoup(html, 'lxml')

In [ ]:
# El objeto soap contiene todo el html

print(soup.prettify())


Con los distintos métodos de la librería nos podremos ir moviendo por la estrucutra del documento. Por ejemplo, podemos ir moviéndonos nivel a nivel accediendo a los hijos de una etiqueta determinada.

In [ ]:
# Llamando a los hijos del objeto soap accedemos a las etiquetas internas de html
list(soup.children)

In [ ]:

[type(item) for item in soup.children]


Todos los items son objetos de la librería BeautifulSoup.

El objeto `Doctype` contiene información acerca del tipo de documento. El objeto `Tag` contendrá en su interior el resto de etiquetas. El objeto NavigableString representa texto encontrado en el documento HTML.

El objeto Tag es el más importante y nos permitirá navegar a través de todo el documento HTML y extraer otras etiquetas y texto. Podemos seleccionar el objeto `Tag` seleccinando el segundo elemento de los hijos del objeto `soup`.

In [ ]:
# Elemento 0 es Doctype
# Elemento 1 tipo Tag
html_tag = list(soup.children)[1]

print(html_tag)

El objeto `html_tag` es también un objeto de tipo ' Tag', por lo que podremos también navegar a través de él.

### <font color="#004D7F"> <i class="fa fa-book" aria-hidden="true" style="color:#113D68"></i> Demo</font>
Vamos a ver como se podría obtener el texto dentro de la etiqueta &lt;p&gt;. Para ello navegaremos hasta la etiqueta y utilizaremos el método `get_text()`.

In [ ]:
# Navegaremos por los children hasta llegar a <p> y despues llamamos a get_text
# Listamos los hijos de la variable anterior 'html_tag' y contamos hasta el <body>
body_tag = list(html_tag.children)[3]

# ahora extraemos los hijos del body tag y contamos hasta llegar a <p> 0 es \n y 1 <p>
parrafo_tag = list(body_tag.children)[1]
    
# ahora obtenemos el texto y lo imprimimos por pantalla
print(parrafo_tag.get_text())

### <font color="#004D7F">Obteniendo las etiquetas de un tipo determinado</font>

Lo que acabamos de ver es útil para ver como navegar por una página, pero implica varios comandos para hacer algo realmente simple. Cuando queremos extraer una etiqueta, podemos utilizar el método `find` para obtener la primera etiqueta de un tipo determinado o `find_all` para obtener la lista completa de etiquetas.

In [ ]:
print(soup.find('p'))

In [ ]:
print(soup.find_all('p'))

### <font color="#004D7F">Buscando por clase e id</font>

Las propiedades class e id fueron introducidas antes. Clases y la id son propiedades que se utilizan para aplicar estilos CSS a determinados elementos HTML. Estas propiedades también se pueden utilizar para especificar sobre qué elementos queremos hacer scraping.

In [ ]:
html='''
<html>
    <head>
        <title>A simple example page</title>
    </head>
    <body>
        <div>
            <p class="inner-text first-item" id="first">
                First paragraph.
            </p>
            <p class="inner-text">
                Second paragraph.
            </p>
        </div>
        <p class="outer-text first-item" id="second">
            <b>
                First outer paragraph.
            </b>
        </p>
        <p class="outer-text" id="third">
            <b>
                Third outer paragraph.
            </b>
        </p>
    </body>
</html>
'''

soup = BeautifulSoup(html, 'lxml')
print('# Clase inner-text')
print(soup.find_all(class_='inner-text'))

print('\n# id: second')
print(soup.find(id='second'))

#print('\n# id: third')
#print(soup.find(id='third'))

### <font color="#004D7F">Selectores CSS</font>

También podemos buscar items usando selectores CSS. Los selectores CSS es la forma en que los desarrolladores web especifican a qué elementos quieren aplicarle estilos CSS. Algunos ejemplos son los siguientes:

* p a: Busca todas las etiquetas a dentro de una etiqueta p
* body p a: Busca todas las etiquetas a dentro de una etiqueta p dentro de la etiqueta body
* html body: Busca todas las etiqueta body dentro del tag html
* p.outer-text: Busca todas las etiqueta p con la clase outer-text
* p#first: Busca todas las etiquetas p con el id 'first'
* body p.outer-text: Busca todas las etiquetas p con la clase outer-text dentro de la etiqueta body

Una buena referencia sobre selectores CSS es la siguiente: [CSS Selectores en Mozilla developers](https:/Selectores en Mozilla developers/developer.mozilla.org/en-US/docs/Learn/CSS/Introduction_to_CSS/Selectors)
You can learn more about CSS selectors here.

En BeautifulSoup se pueden buscar elementos con selectores CSS utilizando el metodo `.select('selector')`.

In [ ]:
soup.select('p.outer-text')
#soup.select('p#third')

In [ ]:
# Ejemplo: Obtener todos los colaboradores de la web ...

# Pasos
# Ver la estructura de la pagina
# 1 mediante las herramientas para desarrolladores del navegador buscamos los div class que se aproximen
# para ello ir navegando por los divs hasta encontrar el child <p>

# Ver en el ejemplo posterior, libreria requests


<div style="text-align: right"> <font size=5> [<i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F">](#indice)</i></font></div>

---

<a id="section5"></a> 
# <font color="#004D7F">5. La librería requests</font>


Lo primero que necesitamos para hacer web scraping es descargar la página. Para ello utilizaremos la libreria requests de Python. La librería hará una petición GET al servidor web, que descargará el HTML.

Además de las llamadas GET, hay otros tipos de llamadas que veremos en el siguiente tutorial.

In [33]:
import requests

page = requests.get("http://www.etics.es")
page

<Response [200]>

Con `page.status_code` obtendremos el código de respuesta de la solicitud. Un código de respuesta 200 significa que todo ha funcionado correctamente.

In [34]:
page.status_code

200

Podemos acceder al contenido de la respuesta, el código HTML, con `page.content`.

In [35]:
page.content

b'<!DOCTYPE html>\n<html lang="ca">\n    <head>\n        <meta name="viewport" content="width=device-width, initial-scale=1.0001, minimum-scale=1.0001, maximum-scale=1.001, shrink-to-fit=no, user-scalable=0">\n        <!-- Forces user OUT of IE\'s compatibility mode and removes "broken page" icon --> \n        <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">\n        <meta name="format-detection" content="telephone=no">\n        <meta http-equiv="x-rim-auto-match" content="none">\n        <meta http-equiv="Content-Type" content="text/html; charset=utf-8" />\n<link rel="shortcut icon" href="https://www.etics.es/sites/all/themes/etics/favicon.ico" type="image/vnd.microsoft.icon" />\n        <title>Home | E-TIC SISTEMES</title>\n        <link type="text/css" rel="stylesheet" href="https://www.etics.es/sites/default/files/css/css_xE-rWrJf-fncB6ztZfd2huxqgxu4WO-qwma6Xer30m4.css" media="all" />\n<link type="text/css" rel="stylesheet" href="https://www.etics.es/sites/default/fil

## <font color="#004D7F">Combiando requests con BeautifulSoup</font>

A continuación se muestra un ejemplo de cómo se podría recoger un HTML con requests y parsearlo con BeautifulSoup.

In [38]:
page = requests.get('https://www.etics.es')
soup = BeautifulSoup(page.content, 'lxml')
for item in soup.select("h1"):
    print(item.get_text())

La nostra experiència, ens avala.
Som més que una Consultora.
Tens un projecte en ment?


---

<a id="section6"></a> 
# <font color="#004D7F">6. Web scraping de una página de predicción meteorológica</font>

### <font color="#004D7F"> <i class="fa fa-book" aria-hidden="true" style="color:#113D68"></i> Demo</font>
Vamos a ver como se podría obtener la temperatura máxima y mínima de hoy en la página [eltiempo.es](http://www.eltiempo.es).


In [ ]:
import requests

# 1 descargamos la pagina donde hariamos el web scraping
page = requests.get('http://www.eltiempo.es/barcelona.html')
# 2 crear el soup y el parser
soup = BeautifulSoup(page.content, 'lxml')
# 3 Mirar la longitud para segurarnos si hay más divs en la pagina con esa clase
# Si el resultado es 1 entonces es que no hay más divs y nos quedamos con este
# div = soup.find_all('div', class_='m_table_weather_day_wrapper')
# print(len(div))
div = soup.find('div', class_='m_table_weather_day_wrapper')
#print(div)

# 4 Seguimos mirando la estructura de la pagina y ver que hay dentro de los divs hasta encontrar la clase que nos 
# interesa y hacemos una busqueda for
for element in div.find_all('div', class_='m_table_weather_day_max_min'):
    span_max = element.find('span', class_='m_table_weather_day_max_temp')
    span_min = element.find('span', class_='m_table_weather_day_min_temp')
    #print(span_min.get_text())

# 5 Ya disponemos del div que contiene las temperaturas y vemos que yo disponen de ningun otro div, sino de span class
# Asi que podemos utilizar la clase span dentro del bucle for

# Nota extra: Si quisieramos sacar el texto a un objeto json

    print({
        "max": span_max.get_text(),
        "min": span_min.get_text()
    })

<div style="text-align: right"> <font size=5> [<i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F">](#indice)</i></font></div>

---

<a id="section7"></a> 
# <font color="#004D7F">7. Web scraping con un navegador real</font>

En algunas ocasiones es posible que no podamos obtener los datos que estamos buscando a través de la librería requests. El principal motivo para que esto ocurra es que el contenido que queremos obtener se renderiza a través de javascript y no podemos obtenerlo solamente cogiendo el código HTML de la página.

Para poder hacer web scraping en páginas renderizadas por javascript necesitamos obtener la página completa (HTML, JS, CSS...), renderizarla en un navegador 'real' y obtener el código HTML resultante después de la ejecución del código javascript.

**Selenium** es una librería en Python que nos permitirá interactuar con un navegador real que tengamos instalado en nuestra máquina (Chrome, Firefox...) o también usar un navegador 'headless' específico. En esta práctica utilizaremos el navegador **phantomJS**.

In [ ]:
!pip install selenium

In [ ]:
from selenium import webdriver
def get_from_phantomJS(url):
    driver = webdriver.PhantomJS()
    driver.get(url)
    text = driver.page_source
    driver.quit()
    return text

print(get_from_phantomJS('http://www.etics.es'))


### <font color="#004D7F"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> Ejercicio</font>

Obtener los datos de demanda de energía eléctrica de la página [https://demanda.ree.es/movil/peninsula/demanda/tablas/1](https://demanda.ree.es/movil/peninsula/demanda/tablas/1).

Para cada fila de la tabla, se debe obtener la fecha/hora y las demandas real, estimada y programada. El resultado final debe ser una lista de diccionarios python. Cada diccionario contendrá las claves `fecha`, `real`, `estimada` y `programada`.

In [ ]:
# Esta pagina se debe hacer con un navegador headless ya que no dispone de codigo html
# Se renderiza completamente mediante javascript por tanto con requests no obtendriamos nada
# Ahora nos quedamos con los div que tienen la fecha, real, estimada y programada

from selenium import webdriver
def get_from_phantomJS(url):
    driver = webdriver.PhantomJS()
    driver.get(url)
    text = driver.page_source
    driver.quit()
    return text

# Add la pagina a una variable
pagina = get_from_phantomJS('https://demanda.ree.es/movil/peninsula/demanda/tablas/1')

# print(get_from_phantomJS('https://demanda.ree.es/movil/peninsula/demanda/tablas/1'))

# Crear objeto soup 
soup = BeautifulSoup(pagina, 'lxml')
# Diccionario
data = []

# Ahora nos vamos a herramientas para desarrolladores y navegamos hasta encontrar la tabla que contiene los datos
# Procedimiento: 
# 1 imprimimos el body (tbody) y buscamos los try td. usamos un bucle for
tbody = soup.find('tbody')
for tr in tbody.find_all('tr'):
    # print(tr)
    # 2 Interesa la informacion que hay dentro de los td
    tds = tr.select('td')
    # print(tds) 
    # Vemos que el primer elemento de la lista devuelto es vacio ya que corresponde a la cabecera y contiene 'th'
    # Los siguientes 4 td son los buenos
    if tds:                   # En python un if de lista vacia evalua a false y no devuelve valor de la primera fila
        element = {
            'fecha': tds[0].get_text(),
            'real': tds[1].get_text(),
            'estimada' : tds[2].get_text(),
            'programada': tds[3].get_text()
        }
        data.append(element)
print(data)




<div style="text-align: left"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#113D68"></i>
 </font></div>

---

<a id="referencias"></a>
# <font color="#004D7F"> Referencias</font>

* [requests](http://docs.python-requests.org/en/master/)
* [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
* [selenium](http://www.seleniumhq.org/)

<div style="text-align: right"> <font size=5> [<i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F">](#indice)</i></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>